In [7]:
# Data Processing Libraries
import pandas as pd
import os
import csv
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

# Model Libraries 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import regularizers

### Load Model Data 
Checkpoint: Load cached data here

In [2]:
# Test Loading Data
model_data_df = pickle.load(open('data/model_data.pkl', 'rb'))
model_data_df

,race,sample_data,label
0,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Metaplastic carcinoma, NOS"
1,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Metaplastic carcinoma, NOS"
2,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
3,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
4,white,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
...,...,...,...
1495,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1496,not reported,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1497,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"
1498,black or african american,"[[0.0, 0.0010240421486530794, 1.0, 0.0, 0.0, 0...","Infiltrating duct carcinoma, NOS"


### Preprocess Data

In [3]:
# ONE HOT ENCODE DATA
encoded_labels = pd.get_dummies(model_data_df.label)
encoded_df = model_data_df.join(encoded_labels)


In [8]:
# Filter Data Frames by Race 
white_df = encoded_df[encoded_df["race"] == "white"]
black_df = encoded_df[encoded_df["race"] == "black or african american"]
asian_df = encoded_df[encoded_df["race"] == "asian"]
prognosis_names = encoded_df.label.unique()

# Train Test Split
def df_split(df): 
    split_index = int(0.7 * len(df)) 
    return df.iloc[:split_index], df.iloc[split_index:]

train_white_df, test_white_df = df_split(white_df) 
train_black_df, test_black_df = df_split(black_df) 
train_asian_df, test_asian_df = df_split(asian_df)

# Concatenate Final Sets 
train_df = pd.concat([train_white_df, train_black_df, train_asian_df])

### Model Results for (train mixed, test black / asian)

In [9]:
# Prepare Model Data --> [takes 1 min. 15 sec.]
def get_x_y(df, prognosis_names): 
    x = np.stack(df.sample_data.values)
    y = df.filter(prognosis_names).values
    return x, y

def train_val_split(data):
    # returns: x_train, x_val, y_train, y_val
    x, y = data
    return train_test_split(x, y, test_size=0.15, random_state=42)


##### Main Code #########
x_train, x_val, y_train, y_val = train_val_split(get_x_y(train_df, prognosis_names)) # training data

x_test_white, y_test_white = get_x_y(test_black_df, prognosis_names)
x_test_black, y_test_black = get_x_y(test_black_df, prognosis_names)
x_test_asian, y_test_asian = get_x_y(test_asian_df, prognosis_names)

In [10]:
# TRAIN NEURAL NETWORK
num_classes = model_data_df.label.nunique()

# Basic DNN Model 
reg_rate = 0.1
model = Sequential([
    Flatten(input_shape=(60623, 26)),  # Flattening the input
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(reg_rate)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(reg_rate)),
    Dense(num_classes, activation='softmax')  # Output layer with softmax for multi-class
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val))


Epoch 1/20
26/26 [==============================] - 41s 1s/step - loss: 125.0574 - accuracy: 0.4747 - val_loss: 61.5682 - val_accuracy: 0.6993
Epoch 2/20
26/26 [==============================] - 38s 1s/step - loss: 59.2514 - accuracy: 0.5204 - val_loss: 54.3444 - val_accuracy: 0.6993
Epoch 3/20
26/26 [==============================] - 43s 2s/step - loss: 40.4722 - accuracy: 0.5933 - val_loss: 26.8635 - val_accuracy: 0.6923
Epoch 4/20
26/26 [==============================] - 42s 2s/step - loss: 23.0151 - accuracy: 0.6489 - val_loss: 16.3254 - val_accuracy: 0.7063
Epoch 5/20
26/26 [==============================] - 37s 1s/step - loss: 19.3822 - accuracy: 0.6069 - val_loss: 14.3102 - val_accuracy: 0.6993
Epoch 6/20
26/26 [==============================] - 41s 2s/step - loss: 14.4028 - accuracy: 0.5946 - val_loss: 11.9538 - val_accuracy: 0.6503
Epoch 7/20
26/26 [==============================] - 39s 1s/step - loss: 12.6150 - accuracy: 0.5773 - val_loss: 14.6824 - val_accuracy: 0.7063
Epoch

In [11]:
# Check Test Performance - White Patients
evaluation = model.evaluate(x_test_white, y_test_white)

# 'evaluation' will contain the loss value and accuracy
loss = evaluation[0]
accuracy = evaluation[1]

print("Performance on White Patients")
print("----------------------------------")
print(f"Loss on test data: {loss}")
print(f"Accuracy on test data: {accuracy}")


3/3 [==============================] - 0s 69ms/step - loss: 3.6573 - accuracy: 0.8205
Performance on White Patients
----------------------------------
Loss on test data: 3.6573004722595215
Accuracy on test data: 0.8205128312110901


In [12]:
# Check Test Performance - Black & AA Patients
evaluation = model.evaluate(x_test_black, y_test_black)

# 'evaluation' will contain the loss value and accuracy
loss = evaluation[0]
accuracy = evaluation[1]

print("Performance on Black Patients")
print("----------------------------------")
print(f"Loss on test data: {loss}")
print(f"Accuracy on test data: {accuracy}")


3/3 [==============================] - 0s 71ms/step - loss: 3.6573 - accuracy: 0.8205
Performance on Black Patients
----------------------------------
Loss on test data: 3.6573004722595215
Accuracy on test data: 0.8205128312110901


In [13]:
# Check Test Performance - Asian Patients
evaluation = model.evaluate(x_test_asian, y_test_asian)

# 'evaluation' will contain the loss value and accuracy
loss = evaluation[0]
accuracy = evaluation[1]

print("Performance on Asian Patients")
print("----------------------------------")
print(f"Loss on test data: {loss}")
print(f"Accuracy on test data: {accuracy}")


1/1 [==============================] - 0s 244ms/step - loss: 3.1354 - accuracy: 0.8214
Performance on Asian Patients
----------------------------------
Loss on test data: 3.1354243755340576
Accuracy on test data: 0.8214285969734192
